In [ ]:
import re
import requests
from bs4 import BeautifulSoup
from lxml import html
import pandas as pd
import dateparser
import geocoder

In [ ]:
all_items = []

In [ ]:
r = requests.get('http://www.mudah.my/', 
                 headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2228.0 Safari/537.36'})
tree = html.fromstring(r.content)
links = tree.xpath("/html/body[@id='simple']/div[@id='homepage-wrap']/div[@id='wrap']/map/area[@class='trackable']//@href")

for state_link in ["http://www.mudah.my/Malaysia/Electronics-for-sale-3000","http://www.mudah.my/Malaysia/Home-Appliances-and-Kitchen-for-sale-4200"]:
    print("NOW CRAWLING FOR {}".format(state_link))
    state_page = requests.get(state_link, 
                     headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.113 Safari/537.36'})
    state_tree = html.fromstring(state_page.content)
    max_page = re.search(r'o=(\d+)',state_tree.xpath("//a[@title='Go to last page']//@href")[0]).group(1)
    for num in range(0+1, int(max_page)+1):
        state_page = requests.get('{}?o={}'.format(state_link,num), 
                         headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.113 Safari/537.36'})
        state_tree = html.fromstring(state_page.content)
        item_pages = state_tree.xpath("//h2[@class='list_title']//@href")
        for page in item_pages:
            try:
                item_page = requests.get(page, 
                                 headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.113 Safari/537.36'})
                item_tree = html.fromstring(item_page.content)
                item_info = item_tree.xpath("//script[contains(text(),'utag')]")
                es = item_info[0]
                item_dict = {k:v for k,v in re.findall('utag_data\["([^\"]*)"\]="([^\"]*)"',es.text)}
                description = item_tree.xpath("//p[@itemprop='description']/text()")
                if description:
                    item_dict['description'] = description[0].strip()
                list_time = item_tree.xpath("//div[@class='list_time']/text()")
                if list_time:
                    item_dict['list_time'] = dateparser.parse(list_time[0].strip()).isoformat()
                all_items.append(item_dict)
            except (KeyboardInterrupt, SystemExit):
                raise
            except:
                print('some error at page {}'.format(page))

In [44]:
import pandas as pd

In [ ]:
df = pd.DataFrame(all_items)

In [ ]:
df.to_excel('Mudah_20170917.xlsx',index=False)

In [145]:
region_geo_df = df[['subregion_name','region_name']].drop_duplicates()

In [ ]:
def getGeoCode(subregion, region):
    g = geocoder.google('{}, {}, Malaysia'.format(subregion, region))
    if g.geojson['features']:
        print(g.geojson['features'])
        return (g.geojson['features'][0]['properties']['lat'], g.geojson['features'][0]['properties']['lng'])

In [ ]:
region_geo_df['coordinate'] = region_geo_df.apply(lambda x: getGeoCode(x['subregion_name'], x['region_name']), axis=1)

In [ ]:
region_geo_df.loc[region_geo_df['subregion_name']=='Others','coordinate'] = None

In [153]:
geo = pd.read_excel('Geolocation.xlsx')
geo['coordinate'] = geo['coordinate'].map(lambda x: tuple(x.split(',')))

In [113]:
region_geo_df2 = region_geo_df.merge(geo, how='outer', on=['subregion_name','region_name'])
region_geo_df2['coordinate'] = region_geo_df2.apply(lambda x: x['coordinate_x'] if pd.notnull(x['coordinate_x']) else x['coordinate_y'], 
                     axis=1)
region_geo_df2 = region_geo_df2.drop(['coordinate_x','coordinate_y'],axis=1)

In [123]:
region_geo_df2['subregion_lat'] = region_geo_df2['coordinate'].map(lambda x: x[0])
region_geo_df2['subregion_lng'] = region_geo_df2['coordinate'].map(lambda x: x[1])
region_geo_df2 = region_geo_df2.drop('coordinate',axis=1)
region_lat = region_geo_df2[region_geo_df2['subregion_name']=='Others'].set_index('region_name')['subregion_lat']
region_lng = region_geo_df2[region_geo_df2['subregion_name']=='Others'].set_index('region_name')['subregion_lng']
region_geo_df2['region_lat'] = region_geo_df2['region_name'].map(region_lat)
region_geo_df2['region_lng'] = region_geo_df2['region_name'].map(region_lng)
region_geo_df2[['subregion_lat','subregion_lng','region_lat','region_lng']] = region_geo_df2[['subregion_lat','subregion_lng','region_lat','region_lng']].applymap(float)

In [174]:
df_final = df.merge(region_geo_df2, how='left', on=['subregion_name','region_name'])

In [176]:
df_final.to_excel('Mudah_20170917.xlsx',index=False)

In [160]:
region_geo_df = df_final[['subregion_name','region_name','subregion_lat', 'subregion_lng', 'region_lat', 'region_lng']].drop_duplicates()

In [157]:
region_geo_df2 = geo

In [158]:
region_geo_df2['subregion_lat'] = region_geo_df2['coordinate'].map(lambda x: x[0])
region_geo_df2['subregion_lng'] = region_geo_df2['coordinate'].map(lambda x: x[1])
region_geo_df2 = region_geo_df2.drop('coordinate',axis=1)
region_lat = region_geo_df2[region_geo_df2['subregion_name']=='Others'].set_index('region_name')['subregion_lat']
region_lng = region_geo_df2[region_geo_df2['subregion_name']=='Others'].set_index('region_name')['subregion_lng']
region_geo_df2['region_lat'] = region_geo_df2['region_name'].map(region_lat)
region_geo_df2['region_lng'] = region_geo_df2['region_name'].map(region_lng)
region_geo_df2[['subregion_lat','subregion_lng','region_lat','region_lng']] = region_geo_df2[['subregion_lat','subregion_lng','region_lat','region_lng']].applymap(float)

In [163]:
region_geo_df3 = region_geo_df.merge(region_geo_df2, 'outer',['subregion_name','region_name'])

In [165]:
region_geo_df3['subregion_lat'] = region_geo_df3.apply(lambda x: x['subregion_lat_y'] if pd.notnull(x['subregion_lat_y']) else x['subregion_lat_x'],  axis=1)
region_geo_df3['subregion_lng'] = region_geo_df3.apply(lambda x: x['subregion_lng_y'] if pd.notnull(x['subregion_lng_y']) else x['subregion_lng_x'],  axis=1)

In [169]:
region_geo_df3 = region_geo_df3.drop(['subregion_lat_x', 'subregion_lng_x',
       'region_lat_x', 'region_lng_x', 'subregion_lat_y', 'subregion_lng_y',
       'region_lat_y', 'region_lng_y'],axis=1)

In [170]:
region_geo_df3['region_lat'] = region_geo_df3['region_name'].map(region_lat)
region_geo_df3['region_lng'] = region_geo_df3['region_name'].map(region_lng)

In [172]:
region_geo_df3[['subregion_lat','subregion_lng','region_lat','region_lng']] = region_geo_df3[['subregion_lat','subregion_lng','region_lat','region_lng']].applymap(float)